# Instalar librería

In [1]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.4 MB/s eta 0:00:00


# Inicialización

In [2]:
import operator
import random
from deap import base, creator, tools, gp, algorithms

# Definición de tipos que usaremos
BIT = bool
INT = int

# Definición de nuestras funciones de puerta lógica
def AND(x, y):
    return x and y

def OR(x, y):
    return x or y

def XOR(x, y):
    return (x and not y) or (not x and y)

def NOT(x):
    return not x

def carry(x, y):
    return x and y

# Conversión de bits a integer
def compose(b0, b1, b2, b3):
    return int(b0) + 2 * int(b1) + 4 * int(b2) + 8 * int(b3)

# Creación del conjunto de primitivos o inputs
pset = gp.PrimitiveSetTyped("MAIN", [BIT] * 6, INT)

# Agregar nuestras funciones al conjunto de primitivos
pset.addPrimitive(AND, [BIT, BIT], BIT)
pset.addPrimitive(OR, [BIT, BIT], BIT)
pset.addPrimitive(XOR, [BIT, BIT], BIT)
pset.addPrimitive(NOT, [BIT], BIT)
pset.addPrimitive(carry, [BIT, BIT], BIT)
pset.addPrimitive(compose, [BIT, BIT, BIT, BIT], INT)

# Clases de individuos y fitness
# El problema es de minimización del error
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

# Los individuos serán árboles
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

# Función de evaluación

In [3]:
# Para cada individuo probamos cada una de las 64 combinaciones de 2 números de 3 bits
def evalAdder(individual):
    func = toolbox.compile(expr=individual)
    error = 0
    for a in range(8):
        for b in range(8):
            # Extraemos los bits desde el menos significativo
            a_bits = [(a >> i) & 1 for i in range(3)]
            b_bits = [(b >> i) & 1 for i in range(3)]
            # Conversión de tipos a booleano
            a_bits = [bool(bit) for bit in a_bits]
            b_bits = [bool(bit) for bit in b_bits]
            try:
                result = func(a_bits[0], a_bits[1], a_bits[2],
                              b_bits[0], b_bits[1], b_bits[2])
            except Exception:
                return 1000,  # Individuos que causan errores son penalizados
            expected = a + b
            error += abs(result - expected)
    return error,

# Registramos la función de evaluación y otras funciones
toolbox.register("evaluate", evalAdder)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# Límite de altura de los árboles es 12
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=12))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=12))

#Main

In [4]:
def main():
    random.seed(382653)
    # Número inicial de individuos
    pop = toolbox.population(n=1300)
    # Calculamos el mejor individuo
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", min)
    stats.register("avg", lambda fits: sum(f[0] for f in fits) / len(fits))

    # Corremos el algoritmo de programación genética
    pop, log = algorithms.eaSimple(pop, toolbox,
                                   cxpb=0.5, mutpb=0.3, ngen=60,
                                   stats=stats, halloffame=hof, verbose=True)

    # Imprimimos el mejor individuo
    print("\nBest individual:")
    print(hof[0])
    print("Fitness:", hof[0].fitness.values[0])

if __name__ == "__main__":
    main()

gen	nevals	min     	avg    
0  	1300  	(128.0,)	390.048
1  	834   	(128.0,)	339.479
2  	811   	(128.0,)	270.5  
3  	840   	(128.0,)	222.685
4  	800   	(116.0,)	201.4  
5  	848   	(88.0,) 	189.492
6  	874   	(88.0,) 	179.443
7  	836   	(84.0,) 	170.228
8  	847   	(84.0,) 	165.735
9  	880   	(84.0,) 	164.363
10 	848   	(84.0,) 	158.218
11 	861   	(84.0,) 	151.065
12 	813   	(84.0,) 	144.736
13 	834   	(84.0,) 	139.893
14 	818   	(84.0,) 	135.766
15 	833   	(82.0,) 	133.589
16 	795   	(80.0,) 	127.678
17 	842   	(82.0,) 	132.967
18 	872   	(82.0,) 	125.493
19 	829   	(81.0,) 	128.315
20 	876   	(81.0,) 	122.575
21 	860   	(80.0,) 	118.974
22 	862   	(78.0,) 	117.668
23 	819   	(78.0,) 	110.047
24 	832   	(78.0,) 	110.925
25 	864   	(80.0,) 	107.447
26 	821   	(80.0,) 	105.769
27 	858   	(78.0,) 	103.042
28 	802   	(78.0,) 	100.583
29 	829   	(78.0,) 	99.6654
30 	836   	(78.0,) 	103.136
31 	852   	(78.0,) 	96.4215
32 	859   	(76.0,) 	98.1777
33 	833   	(78.0,) 	94.74  
34 	863   	(76.0,) 	